## Init And Test Alger

In [ ]:
from openrouteservice import Client



client = Client(key='eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6Ijk5YTY0MDIxMjAzYjRmOGI5YmY0OGExNzkyODI2OGYxIiwiaCI6Im11cm11cjY0In0=')



In [14]:
import folium

# Create a list of coordinates for the points
points = [
    [36.775, 3.05],
    [36.77, 3.06],
    [36.78, 3.055],
    [36.76, 3.04]
]

map = folium.Map(location=[36.77314237420313, 3.059924163605557], zoom_start=13)

# Add markers for each point
for point in points:
    folium.Marker(location=point).add_to(map)

from IPython.display import display
display(map)

## Blida Data and Graph

In [ ]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt


# Define the area you want to model
place_name = "Blida, Algeria"

# Download the street network for the area
# This returns a networkx MultiDiGraph
#print(f"Downloading map data for {place_name}...")
#G = ox.graph_from_place(place_name, network_type='drive')
#print("Map data downloaded.")
# You can save the graph to avoid re-downloading
#ox.save_graphml(G, 'pune_map.graphml')

# Load the graph
G = ox.load_graphml(r'C:\Users\ABDO\Desktop\- LeanSpace\TraficRouting\App\data\Blida_map.graphml')



num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")



Number of nodes: 2662
Number of edges: 6916


: 

In [ ]:
station_start_coords = (36.4700, 2.8000) 
station_end_coords = (36.4800, 2.8100)

# Find nearest nodes
start_node = ox.distance.nearest_nodes(G, X=station_start_coords[1], Y=station_start_coords[0])
end_node = ox.distance.nearest_nodes(G, X=station_end_coords[1], Y=station_end_coords[0])

print(f"Start Node ID: {start_node}")
print(f"End Node ID: {end_node}")

# --- Plotting the Graph (Nodes and Edges) ---

# Create a figure
plt.figure(figsize=(12, 8))

# Convert the MultiDiGraph to an undirected Graph for simpler plotting (optional)
# This avoids plotting both directions of each edge
G_simple = nx.Graph(G)  # This merges multi-edges and ignores direction

# Get positions for the nodes (using the actual latitude/longitude)
pos = {node: (data['x'], data['y']) for node, data in G_simple.nodes(data=True)}

# Draw edges
nx.draw_networkx_edges(G_simple, pos, edge_color='gray', width=1)

# Draw all nodes
nx.draw_networkx_nodes(G_simple, pos, node_size=10, node_color='blue', alpha=0.7)

# Highlight start and end nodes
nx.draw_networkx_nodes(G_simple, pos, nodelist=[start_node], node_size=100, node_color='green', label='Start')
nx.draw_networkx_nodes(G_simple, pos, nodelist=[end_node], node_size=100, node_color='red', label='End')

# Optional: Add labels for start/end
# plt.annotate('Start', xy=pos[start_node], xytext=(5, 5), textcoords='offset points', fontsize=12, color='green')
# plt.annotate('End', xy=pos[end_node], xytext=(5, 5), textcoords='offset points', fontsize=12, color='red')

# Remove axes for clarity
plt.axis('off')
plt.title("Street Network Graph (Nodes and Edges)", fontsize=16)

# Add legend manually since we used different node colors
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='blue', label='Nodes'),
    Patch(facecolor='green', label='Start Node'),
    Patch(facecolor='red', label='End Node')
]
plt.legend(handles=legend_elements, loc='upper right')

# Show the plot
plt.tight_layout()
plt.show()

In [10]:
import folium

map = folium.Map(location=[36.4700, 2.8000], zoom_start=13)

folium.Marker(location=station_start_coords ).add_to(map)
from IPython.display import display
display(map)

## A* Route

In [13]:
# Calculate the shortest path using the A* algorithm
# The 'weight' parameter tells the algorithm which edge attribute to minimize.
# 'length' will give the shortest route in terms of distance.
route = nx.astar_path(G, start_node, end_node, weight='length')

print(f"Route found with {len(route)} nodes.")

# Create a folium map centered on the start node.
# The old ox.plot_route_folium function is deprecated.
# We now create the map and add the route manually.
route_map = folium.Map(location=[G.nodes[start_node]['y'], G.nodes[start_node]['x']], zoom_start=15)

# get the geometry of the route
route_nodes = G.nodes(data=True)
route_line = [(route_nodes[node]['y'], route_nodes[node]['x']) for node in route]

# add the route as a red line to the map
folium.PolyLine(route_line, color='red', weight=3).add_to(route_map)

# Save the map to an HTML file
route_map.save('pune_route.html')

print("Route map saved to pune_route.html")

# Display the map
route_map

Route found with 19 nodes.
Route map saved to pune_route.html
